In [4]:
import pandas as pd
import numpy as np
import json
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import sys

import pickle


%matplotlib inline

In [5]:
rand_seed = 0  # random state for reproducibility
np.random.seed(rand_seed)
# reading our prepared data
data = pd.read_excel('ALL_data.xlsx')
data = data.dropna()
data.head()

,comment,label
0,نفسي يوم تكتبو السعر بدون مانسال,سلبي
1,طيب ما تشرحو طريقه الاشتراك في الباقه دي,محايد
2,لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...,سلبي
3,رمز الاشتراك شنو,محايد
4,واو,ايجابي


In [4]:
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data

In [5]:
train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
val_fraction = .50   # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = data.columns.tolist() # the features columns
features.remove(output)
print('output:', output)
print('features:', features)

train_data, tmp = random_split(data, features, output, train_fraction, rand_seed)
val_data, test_data = random_split(tmp, features, output, val_fraction, rand_seed)

print("train data = "+str(len(train_data)))
print("val  data = "+str(len(val_data)))
print("test  data = "+str(len(test_data)))

print("all data = "+str(len(data)))

output: label
features: ['comment']
train data = 2458
val  data = 307
test  data = 308
all data = 3073


In [24]:
tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, max_df=0.5, stop_words=None, use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['comment'].values.astype('U'))
val_data_features_tfidf= tfidfvectorizer.transform(val_data['comment'].values.astype('U'))
test_data_features_tfidf = tfidfvectorizer.transform(test_data['comment'].values.astype('U'))


In [25]:
train_data_features_tfidf.shape, val_data_features_tfidf.shape, test_data_features_tfidf.shape

((2458, 18211), (307, 18211), (308, 18211))

In [26]:
def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    val_data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='result_'+str(clf)[0:15]+'.xlsx'
    val_data.to_excel(filename)

In [27]:
logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9214808787632222
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7328990228013029
f1_score  on test data:
0.6620345828600277


In [33]:
tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['comment'])
val_data_features_tfidf= tfidfvectorizer.transform(val_data['comment'])
test_data_features_tfidf = tfidfvectorizer.transform(test_data['comment'])


In [34]:
train_data_features_tfidf.shape, val_data_features_tfidf.shape, test_data_features_tfidf.shape

((2458, 30173), (307, 30173), (308, 30173))

In [35]:
logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9141578519121237
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7198697068403909
f1_score  on test data:
0.6345022652749618


In [39]:
tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['comment'])
val_data_features_tfidf= tfidfvectorizer.transform(val_data['comment'])
test_data_features_tfidf = tfidfvectorizer.transform(test_data['comment'])


In [40]:
train_data_features_tfidf.shape, val_data_features_tfidf.shape, test_data_features_tfidf.shape

((2458, 5799), (307, 5799), (308, 5799))

In [41]:
logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9092758340113913
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.749185667752443
f1_score  on test data:
0.6906799203828907


In [45]:
bowvectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features = 1000)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_bow = bowvectorizer.fit_transform(train_data['comment'])
val_data_features_bow= bowvectorizer.transform(val_data['comment'])
test_data_features_bow = bowvectorizer.transform(test_data['comment'])


In [46]:
train_data_features_bow.shape, val_data_features_bow.shape, test_data_features_bow.shape

((2458, 1000), (307, 1000), (308, 1000))

In [47]:
logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_bow, train_data[output],
                        val_data_features_bow, val_data[output],val_data)

accuracy_score Score on training data:
0.8730675345809601
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7296416938110749
f1_score  on test data:
0.6916641779848419


In [2]:
from aranorm import normalize_arabic_text
from nltk.corpus import stopwords
from nltk import word_tokenize
import pandas as pd



In [7]:

#Data Cleaning

clean_data = data

clean_data.shape

(3073, 2)

In [64]:
clean_data['clean']=normalize_arabic_text(clean_data['comment'])


TypeError: expected string or bytes-like object

In [67]:
clean_data['clean']=clean_data['clean'].apply(lambda x: ' '.join([w for comment in normalize_arabic_text(clean_data['clean'])] ))

TypeError: expected string or bytes-like object

In [77]:
for i in range(0,len(clean_data)):

    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
   

In [78]:
for i in range(0,len(clean_data)):

    print(clean_data['comment']+"  "+clean_data['clean'])

0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...           

Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                

0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...           

Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                

Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                

...

Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                

0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...           

0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...           

Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                

0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...                        
3068                             خليك سوداني  خليك سوداني
3069                                       سوداني  سوداني
3070         سوداني الاقوي والافضل  سوداني الاقوي والافضل
3071                             خليك سوداني  خليك سوداني
3072                                       سوداني  سوداني
Length: 3073, dtype: object
0       نفسي يوم تكتبو السعر بدون مانسال  نفسي يوم تكت...
1       طيب ما تشرحو طريقه الاشتراك في الباقه دي  طيب ...
2        لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
3                      رمز الاشتراك شنو  رمز الاشتراك شنو
4                                                واو  واو
                              ...           

In [79]:
clean_data['clean']

0                        نفسي يوم تكتبو السعر بدون مانسال
1                طيب ما تشرحو طريقه الاشتراك في الباقه دي
2       لو قللتو الرساءل دي واديتونا ليها ميقات يكون اجمل
3                                        رمز الاشتراك شنو
4                                                     واو
                              ...                        
3068                                          خليك سوداني
3069                                               سوداني
3070                                سوداني الاقوي والافضل
3071                                          خليك سوداني
3072                                               سوداني
Name: clean, Length: 3073, dtype: object

In [80]:
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data

In [87]:
train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
val_fraction = .50   # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = clean_data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
print('output:', output)
print('features:', features)

train_data, tmp = random_split(data, features, output, train_fraction, rand_seed)
val_data, test_data = random_split(tmp, features, output, val_fraction, rand_seed)

print("train data = "+str(len(train_data)))
print("val  data = "+str(len(val_data)))
print("test  data = "+str(len(test_data)))

print("all data = "+str(len(data)))

output: label
features: ['clean']
train data = 2458
val  data = 307
test  data = 308
all data = 3073


In [88]:
tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['clean'])
val_data_features_tfidf= tfidfvectorizer.transform(val_data['clean'])
test_data_features_tfidf = tfidfvectorizer.transform(test_data['clean'])


In [89]:
logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9072416598860863
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.742671009771987
f1_score  on test data:
0.6867818862034932


In [121]:
import gensim
Tokenize_comment =clean_data['clean'].apply(lambda x: x.split())

Model_W2V = gensim.models.Word2Vec(Tokenize_comment,# No. of features
                                   window=5, 
                                   min_count=2, 
                                   sg=1, # 1 for skip-gram model
                                   hs=0,
                                   negative=10, # for negative sampling
                                   workers=2,  # No. of cores
                                   seed=34 )

Model_W2V.train(Tokenize_comment, total_examples=len(clean_data['clean']), epochs=20)

(305771, 449440)

In [124]:
def word_vec(tokens, size):
  vec = np.zeros(size).reshape((1, size))
  count = 0
  for word in tokens:
    try:
      vec += Model_W2V.wv[word].reshape((100,size))
      count += 1
    except KeyError:
      
      continue
    
  if count != 0:
    vec = vec/count
   
  return vec
     

Arrays = np.zeros((len(Tokenize_comment), 200))

for i in range(len(Tokenize_comment)):
  Arrays[i,:] = word_vec(Tokenize_comment[i], 200)
  
Arr_df = pd.DataFrame(Arrays)
Arr_df.shape
     

ValueError: cannot reshape array of size 100 into shape (100,200)

In [110]:

word_vectors =Model_W2V.wv

In [102]:
Model_W2V.wv.most_similar(positive="سعر")
     

[('وسعر', 0.8724102973937988),
 ('الباقات', 0.8597978949546814),
 ('رافعين', 0.8256046175956726),
 ('جديد', 0.8019540309906006),
 ('الدولار', 0.7934174537658691),
 ('رفعتو', 0.7879610657691956),
 ('يزيدو', 0.7795857191085815),
 ('وشكرا', 0.7728636860847473),
 ('يومين', 0.7655701041221619),
 ('كلما', 0.7647611498832703)]

In [116]:
Model_W2V

In [120]:
def word_vec(tokens, size):
  vec = np.zeros(size).reshape((1, size))
  count = 0
  for word in tokens:
    try:
      vec += Model_W2V.wv[word].reshape((1, size))
      count += 1
    except KeyError:
      
      continue
    
  if count != 0:
    vec = vec/count
   
  return vec
     

Arrays = np.zeros((len(Tokenize_comment), 200))

for i in range(len(Tokenize_comment)):
  Arrays[i,:] = word_vec(Tokenize_comment[i], 200)
  
Arr_df = pd.DataFrame(Arrays)
Arr_df.shape
     

ValueError: cannot reshape array of size 100 into shape (1,200)

In [114]:
len(word_vectors['سعر'])

100

In [127]:
from nltk.corpus import stopwords
from nltk import word_tokenize

for i in range(0,len(clean_data)):
    tokenizedRow = word_tokenize(clean_data['clean'][i])
    commentWithNoStopWords= ' '.join([i for i in tokenizedRow if i not in arabicStopWords])
                

    clean_data['clean'][i]=commentWithNoStopWords
    
    
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data
   

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\rabab/nltk_data'
    - 'C:\\Users\\rabab\\anaconda3\\nltk_data'
    - 'C:\\Users\\rabab\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\rabab\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\rabab\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [128]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rabab\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [129]:
from nltk.corpus import stopwords
from nltk import word_tokenize

for i in range(0,len(clean_data)):
    tokenizedRow = word_tokenize(clean_data['clean'][i])
    commentWithNoStopWords= ' '.join([i for i in tokenizedRow if i not in arabicStopWords])
                

    clean_data['clean'][i]=commentWithNoStopWords
    
    
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data
   

NameError: name 'arabicStopWords' is not defined

In [130]:
arabicStopWords= stopwords.words("arabic")

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\rabab/nltk_data'
    - 'C:\\Users\\rabab\\anaconda3\\nltk_data'
    - 'C:\\Users\\rabab\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\rabab\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\rabab\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [131]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rabab\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [132]:
arabicStopWords= stopwords.words("arabic")

In [133]:
from nltk.corpus import stopwords
from nltk import word_tokenize

for i in range(0,len(clean_data)):
    tokenizedRow = word_tokenize(clean_data['clean'][i])
    commentWithNoStopWords= ' '.join([i for i in tokenizedRow if i not in arabicStopWords])
                

    clean_data['clean'][i]=commentWithNoStopWords
    
    
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data
   

In [134]:
train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
val_fraction = .50   # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = clean_data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
print('output:', output)
print('features:', features)

train_data, tmp = random_split(data, features, output, train_fraction, rand_seed)
val_data, test_data = random_split(tmp, features, output, val_fraction, rand_seed)

print("train data = "+str(len(train_data)))
print("val  data = "+str(len(val_data)))
print("test  data = "+str(len(test_data)))

print("all data = "+str(len(data)))

output: label
features: ['clean']
train data = 2458
val  data = 307
test  data = 308
all data = 3073


In [135]:
tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['clean'])
val_data_features_tfidf= tfidfvectorizer.transform(val_data['clean'])
test_data_features_tfidf = tfidfvectorizer.transform(test_data['clean'])


In [136]:
logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9092758340113913
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7654723127035831
f1_score  on test data:
0.7162021948907196


In [142]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import itertools
import re

def remove_duplicated_words(text):
    s1=[]
    text=' '.join(k for k,v in itertools.groupby(text.replace("&lt;/Sent&gt;","").split()))
    s=re.sub(r'b(.+)(\s+\b)+',r'\1',text)
    return ''.join(s)
    
    
for i in range(0,len(clean_data)):
    commentWithNoDuplicated = remove_duplicated_words(clean_data['clean'][i])
                

    clean_data['clean'][i]=commentWithNoDuplicated
    
    
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data

train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
val_fraction = .50   # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = clean_data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
print('output:', output)
print('features:', features)

train_data, tmp = random_split(data, features, output, train_fraction, rand_seed)
val_data, test_data = random_split(tmp, features, output, val_fraction, rand_seed)

print("train data = "+str(len(train_data)))
print("val  data = "+str(len(val_data)))
print("test  data = "+str(len(test_data)))

print("all data = "+str(len(data)))

tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['clean'])
val_data_features_tfidf= tfidfvectorizer.transform(val_data['clean'])
test_data_features_tfidf = tfidfvectorizer.transform(test_data['clean'])

logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

output: label
features: ['clean']
train data = 2458
val  data = 307
test  data = 308
all data = 3073
accuracy_score Score on training data:
0.9096826688364524
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7654723127035831
f1_score  on test data:
0.7162021948907196


In [143]:
pip install stanza


  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=d9f83e1bf167ef985c7820569169f6e1f3e54a74228f1ce07b83498701686a56
  Stored in directory: c:\users\rabab\appdata\local\pip\cache\wheels\9a\b8\0f\f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [153]:
import stanza


nlp = stanza.Pipeline(lang='ar', processors='tokenize,lemma')
doc = nlp('انا هو يقال ان كلهم كثيرون')

lema=''.join(word.lemma+' ' for sent in doc.sentences for word in sent.words)
#print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')
print(lema)

2022-12-28 16:09:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-28 16:09:51 WARNING: Language ar package default expects mwt, which has been added
2022-12-28 16:09:51 INFO: Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| lemma     | padt    |

2022-12-28 16:09:51 INFO: Use device: cpu
2022-12-28 16:09:51 INFO: Loading: tokenize
2022-12-28 16:09:51 INFO: Loading: mwt
2022-12-28 16:09:51 INFO: Loading: lemma
2022-12-28 16:09:51 INFO: Done loading processors!


انا هُوَ أَقَال أَنَّ كِلهَم كَثِير 


In [10]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import itertools
import re



def remove_duplicated_words(text):
    s1=[]
    text=' '.join(k for k,v in itertools.groupby(text.replace("&lt;/Sent&gt;","").split()))
    s=re.sub(r'b(.+)(\s+\b)+',r'\1',text)
    return ''.join(s)
    
    
for i in range(0,len(clean_data)):
    doc = nlp(clean_data['clean'][i])
    lema=''.join(word.lemma+' ' for sent in doc.sentences for word in sent.words)

    clean_data['clean'][i]=lema

for i in range(0,len(clean_data)):  
    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data

train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
val_fraction = .50   # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = clean_data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
print('output:', output)
print('features:', features)

train_data, tmp = random_split(data, features, output, train_fraction, rand_seed)
val_data, test_data = random_split(tmp, features, output, val_fraction, rand_seed)

print("train data = "+str(len(train_data)))
print("val  data = "+str(len(val_data)))
print("test  data = "+str(len(test_data)))

print("all data = "+str(len(data)))

tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf = tfidfvectorizer.fit_transform(train_data['clean'])
val_data_features_tfidf= tfidfvectorizer.transform(val_data['clean'])
test_data_features_tfidf = tfidfvectorizer.transform(test_data['clean'])

logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

KeyError: 'clean'

In [160]:
bowvectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features = 1000)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_bow = bowvectorizer.fit_transform(train_data['clean'])
val_data_features_bow= bowvectorizer.transform(val_data['clean'])
test_data_features_bow = bowvectorizer.transform(test_data['clean'])


logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9068348250610252
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7915309446254072
f1_score  on test data:
0.7418745361460181


In [161]:
def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    val_data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='result_'+str(clf)[0:15]+'.xlsx'
    val_data.to_excel(filename)

In [162]:
bowvectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features = 1000)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_bow = bowvectorizer.fit_transform(train_data['clean'])
val_data_features_bow= bowvectorizer.transform(val_data['clean'])
test_data_features_bow = bowvectorizer.transform(test_data['clean'])


logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf, train_data[output],
                        val_data_features_tfidf, val_data[output],val_data)

accuracy_score Score on training data:
0.9068348250610252
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7915309446254072
f1_score  on test data:
0.7418745361460181


In [11]:
# here is lemma with all models
import stanza


nlp = stanza.Pipeline(lang='ar', processors='tokenize,lemma')

def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    val_data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='lemma_'+str(clf)[0:15]+'.xlsx'
    val_data.to_excel(filename)
# L E M M A
for i in range(0,len(clean_data)):
    doc = nlp(clean_data['clean'][i])
    lema=''.join(word.lemma+' ' for sent in doc.sentences for word in sent.words)

    clean_data['clean'][i]=lema
    
# N O R M A L I Z E

for i in range(0,len(clean_data)):  
    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
    
# vectorizer = CountVectorizer(ngram_range=(1, 2))



bowvectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features = 1000)
train_data_features_bow_lemma = bow_lemma_vectorizer.fit_transform(train_data['clean'])
val_data_features_bow_lemma= bow_lemma_vectorizer.transform(val_data['clean'])
test_data_features_bow_lemma = bow_lemma_vectorizer.transform(test_data['clean'])
# B O W 

logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_bow_lemma, train_data[output],
                        val_data_features_bow_lemma, val_data[output],val_data)

# TF IDF 

tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf_lemma = tfidf_lemma_vectorizer.fit_transform(train_data['clean'])
val_data_features_tfidf_lemma= tfidf_lemma_vectorizer.transform(val_data['clean'])
test_data_features_tfidf_lemma = tfidf_lemma_vectorizer.transform(test_data['clean'])

logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf_lemma, train_data[output],
                        val_data_features_tfidf_lemma, val_data[output],val_data)

2022-12-29 05:55:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-29 05:55:23 WARNING: Language ar package default expects mwt, which has been added
2022-12-29 05:55:23 INFO: Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| lemma     | padt    |

2022-12-29 05:55:23 INFO: Use device: cpu
2022-12-29 05:55:23 INFO: Loading: tokenize
2022-12-29 05:55:23 INFO: Loading: mwt
2022-12-29 05:55:23 INFO: Loading: lemma
2022-12-29 05:55:24 INFO: Done loading processors!


KeyError: 'clean'

In [14]:
from aranorm import normalize_arabic_text
for i in range(0,len(clean_data)):

    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
   

KeyError: 'clean'

In [16]:
train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
val_fraction = .50   # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = clean_data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
print('output:', output)
print('features:', features)



output: label
features: []


In [17]:
for i in range(0,len(clean_data)):

    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
   

KeyError: 'clean'

In [19]:

#Data Cleaning

clean_data = data

clean_data.shape



(3073, 2)

In [20]:
clean_data['clean']=clean_data['comment']

In [21]:
for i in range(0,len(clean_data)):

    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
   

In [25]:
# here is lemma with all models
import stanza


nlp = stanza.Pipeline(lang='ar', processors='tokenize,lemma')



2022-12-31 01:20:54 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-31 01:21:11 WARNING: Language ar package default expects mwt, which has been added
2022-12-31 01:21:11 INFO: Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| lemma     | padt    |

2022-12-31 01:21:11 INFO: Use device: cpu
2022-12-31 01:21:11 INFO: Loading: tokenize
2022-12-31 01:21:11 INFO: Loading: mwt
2022-12-31 01:21:11 INFO: Loading: lemma
2022-12-31 01:21:11 INFO: Done loading processors!


In [27]:

def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    val_data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='lemma_'+str(clf)[0:15]+'.xlsx'
    val_data.to_excel(filename)
# L E M M A
for i in range(0,len(clean_data)):
    doc = nlp(clean_data['clean'][i])
    lema=''.join(word.lemma+' ' for sent in doc.sentences for word in sent.words)

    clean_data['clean'][i]=lema
    
# N O R M A L I Z E

for i in range(0,len(clean_data)):  
    clean_data['clean'][i]=normalize_arabic_text(clean_data['clean'][i])
    
    
# vectorizer = CountVectorizer(ngram_range=(1, 2))



In [31]:
def random_split(clean_data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data


train_clean_data, tmp = random_split(clean_data, features, output, train_fraction, rand_seed)
val_clean_data, test_clean_data = random_split(tmp, features, output, val_fraction, rand_seed)



In [34]:

bow_lemma_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features = 1000)
train_data_features_bow_lemma = bow_lemma_vectorizer.fit_transform(train_clean_data['clean'])
val_data_features_bow_lemma= bow_lemma_vectorizer.transform(val_clean_data['clean'])
test_data_features_bow_lemma = bow_lemma_vectorizer.transform(test_clean_data['clean'])
# B O W 

logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_bow_lemma, train_clean_data[output],
                        val_data_features_bow_lemma, val_data[output],val_data)

# TF IDF 

tfidf_lemma_vectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_df=0.5,  use_idf=True)
# vectorizer = CountVectorizer(ngram_range=(1, 2))

train_data_features_tfidf_lemma = tfidf_lemma_vectorizer.fit_transform(train_clean_data['clean'])
val_data_features_tfidf_lemma= tfidf_lemma_vectorizer.transform(val_clean_data['clean'])
test_data_features_tfidf_lemma = tfidf_lemma_vectorizer.transform(test_clean_data['clean'])

logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features_tfidf_lemma, train_clean_data[output],
                        val_data_features_tfidf_lemma, val_clean_data[output],val_data)

NameError: name 'train_clean_data' is not defined